1- loading data set using PyTorch

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader

class ImageLabelDataset(Dataset):
    def __init__(self, root_dir, data_types):
        self.data = {}
        for data_type in data_types:
            data_path = os.path.join(root_dir, data_type)
            image_folder_path = os.path.join(data_path, "images")
            label_folder_path = os.path.join(data_path, "labels", "json")
            images = []
            labels = []
            for image_file in os.listdir(image_folder_path):
                image_path = os.path.join(image_folder_path, image_file)
                json_file = image_file.split('.')[0] + '.json'
                label_path = os.path.join(label_folder_path, json_file)
                if os.path.exists(label_path):
                    images.append(image_path)
                    labels.append(label_path)
            self.data[data_type] = {"images": images, "labels": labels}

    def __len__(self):
        total_images = 0
        for data_type in self.data:
            total_images += len(self.data[data_type]["images"])
        return total_images

    def __getitem__(self, idx):
     data_type = None
     image_idx = None
     for dt, data in self.data.items():
        if idx < len(data['images']):
            data_type = dt
            image_idx = idx
            break
        else:
            idx -= len(data['images'])
     if data_type is not None and image_idx is not None:
        image_path = self.data[data_type]["images"][image_idx]
        label_path = self.data[data_type]["labels"][image_idx]
        return image_path, label_path
     else:
        raise IndexError("Index out of range.")


2- visualizing some of the labeled images

In [ ]:
def visualize_images(data_loader, num_batches):
    colors = {"bin": (255, 0, 0), "dolly": (0, 255, 0), "jack": (0, 0, 255)}
    
    for batch_idx, batch in enumerate(data_loader):
        if batch_idx >= num_batches:
            break
        
        images, labels = batch
        for image_path, label_path in zip(images, labels):
            image_cv2 = cv2.imread(image_path)
            image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(8, 6))
            plt.imshow(image_cv2)

            with open(label_path, 'r') as f:
                labels = json.load(f)

            for bbox in labels:
                left = bbox["Left"]
                top = bbox["Top"]
                right = bbox["Right"]
                bottom = bbox["Bottom"]
                class_name = bbox["ObjectClassName"]
                color = colors.get(class_name, (0, 0, 0))
            

                cv2.rectangle(image_cv2, (left, top), (right, bottom), color, 2)
                cv2.putText(image_cv2, class_name, (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 4)

            plt.imshow(image_cv2)
            plt.title(image_path)
            plt.show()
           


root_dir = "data"
dataset_train = ImageLabelDataset(root_dir, ["Training"])
dataset_test = ImageLabelDataset(root_dir, ["Testing"])
batch_size = 2

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

num_batches_to_visualize = 1
print("Visualizing training images:")
visualize_images(train_loader, num_batches_to_visualize)

print("Visualizing testing images:")
visualize_images(test_loader, num_batches_to_visualize)

3- adding agmentations

In [ ]:
import albumentations as A

def augment_and_save_images(data_loader, output_dir, num_batches=1):
    for batch_idx, batch in enumerate(data_loader):
        if batch_idx >= num_batches:
            break  
        augmentation = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
            A.RandomGamma(gamma_limit=(80, 120), p=0.5),
            A.Blur(blur_limit=(3, 7), p=0.5),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels', 'class_ids']))
        
        images, labels = batch
    
        for image_path, label_path in zip(images, labels):
            image = cv2.imread(image_path)
            filename = os.path.splitext(os.path.basename(image_path))[0] 
            output_img_path = os.path.join(output_dir, f"{filename}_a.jpg") 
            output_label_path = os.path.join(os.path.dirname(label_path), f"{filename}_a.json") 
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    try:
                        original_labels = json.load(f)
                    except json.JSONDecodeError:
                        print(f"Error: Unable to parse JSON file: {label_path}")
                        continue

                bboxes = []
                class_labels = []
                class_ids = []  # Add class IDs list
                for bbox in original_labels:
                    try:
                        class_name = bbox["ObjectClassName"]
                        class_id = bbox["ObjectClassId"]  # Get class ID
                        left = bbox["Left"]
                        top = bbox["Top"]
                        right = bbox["Right"]
                        bottom = bbox["Bottom"]
                    except KeyError:
                        print(f"Error: Malformed label in file: {label_path}")
                        continue
                    bboxes.append([left, top, right, bottom])
                    class_labels.append(class_name)
                    class_ids.append(class_id)  # Append class ID

                augmented = augmentation(image=image, bboxes=bboxes, class_labels=class_labels, class_ids=class_ids)
                augmented_image = augmented['image']
                augmented_bboxes = augmented['bboxes']
                cv2.imwrite(output_img_path, augmented_image)
                with open(output_label_path, 'w') as f_out:
                    augmented_labels = []
                    for bbox, class_id in zip(augmented_bboxes, class_ids):
                        augmented_labels.append({
                            "Left": int(bbox[0]),
                            "Top": int(bbox[1]),
                            "Right": int(bbox[2]),
                            "Bottom": int(bbox[3]),
                            "ObjectClassName": class_labels[augmented_bboxes.index(bbox)],
                            "ObjectClassId": class_id  # Include class ID in augmented JSON
                        })
                    json.dump(augmented_labels, f_out)

root_dir = "data"
output_dir = "data/Training/images"
dataset_train = ImageLabelDataset(root_dir, ["Training"])
batch_size = 3
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
num_batches_to_augment = 1

print("Augmenting and saving training images:")
augment_and_save_images(train_loader, output_dir, num_batches_to_augment)


converting to yolo format

In [ ]:
import os
import json

def convert_to_yolo(bbox):
    x_center = (bbox['Left'] + bbox['Right']) / (2)
    y_center = (bbox['Top'] + bbox['Bottom']) / (2)
    width = (bbox['Right'] - bbox['Left']) 
    height = (bbox['Bottom'] - bbox['Top']) 
    return f"{bbox['ObjectClassId']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

input_folder = "data/Training/labels/json"

output_folder = "data/Training/labels/yolo"
os.makedirs(output_folder, exist_ok=True)


for json_file in os.listdir(input_folder):
    if json_file.endswith('.json'):
        with open(os.path.join(input_folder, json_file), 'r') as f:
            data = json.load(f)
        yolo_data = []
        for bbox in data:
            yolo_data.append(convert_to_yolo(bbox))
        output_filename = os.path.splitext(json_file)[0] + '.txt'
        with open(os.path.join(output_folder, output_filename), 'w') as f:
            f.write('\n'.join(yolo_data))

print("Conversion completed.")


In [ ]:
# normalize
import os
import cv2

def normalize_yolo_labels(image_dir, label_dir):
    for filename in os.listdir(image_dir):
        if filename.endswith((".jpg")):
            image_path = os.path.join(image_dir, filename)
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")

            if os.path.exists(label_path):
                # Read image dimensions
                image = cv2.imread(image_path)
                image_height, image_width, _ = image.shape

                # Read and normalize labels
                with open(label_path, "r") as file:
                    lines = file.readlines()
                    normalized_lines = []
                    for line in lines:
                        class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())

                        # Normalize bounding box coordinates
                        x_center_norm = x_center / image_width
                        y_center_norm = y_center / image_height
                        box_width_norm = box_width / image_width
                        box_height_norm = box_height / image_height

                        normalized_line = f"{int(class_id)} {x_center_norm:.6f} {y_center_norm:.6f} {box_width_norm:.6f} {box_height_norm:.6f}\n"
                        normalized_lines.append(normalized_line)

                # Write normalized labels back to file
                with open(label_path, "w") as file:
                    file.writelines(normalized_lines)

# Example usage
image_directory = "data/Training/images"
label_directory = "data/Training/labels/yolo"
normalize_yolo_labels(image_directory, label_directory)


4- splitting the dataset

In [ ]:
import os
import shutil
import random

def create_directories(base_folder, subfolders):
    for folder in subfolders:
        os.makedirs(os.path.join(base_folder, folder), exist_ok=True)

def copy_files_to_folder(file_list, dest_folder):
    for file_path in file_list:
        shutil.copy(file_path, dest_folder)

def split_dataset(image_folder, label_folder, split_ratio=0.2):
    # Get the list of image files
    image_files = os.listdir(image_folder)
    num_images = len(image_files)

    # Shuffle the image files
    random.shuffle(image_files)

    # Calculate the number of images for validation based on the split ratio
    num_val_images = int(split_ratio * num_images)

    # Split the image files into training and validation sets
    val_image_files = image_files[:num_val_images]
    train_image_files = image_files[num_val_images:]

    # Create training and validation datasets
    train_dataset = {
        "images": [os.path.join(image_folder, img) for img in train_image_files],
        "labels": [os.path.join(label_folder, os.path.splitext(img)[0] + '.txt') for img in train_image_files]
    }
    val_dataset = {
        "images": [os.path.join(image_folder, img) for img in val_image_files],
        "labels": [os.path.join(label_folder, os.path.splitext(img)[0] + '.txt') for img in val_image_files]
    }

    return train_dataset, val_dataset

# Specify the paths to the image and label folders
image_folder = "data/Training/images"
label_folder = "data/Training/labels/yolo"

# Split the dataset with a 80-20 split ratio (80% training, 20% validation)
train_dataset, val_dataset = split_dataset(image_folder, label_folder, split_ratio=0.2)

# Create folders for train and validation datasets
train_folder = "data/Training/train"
val_folder = "data/Training/validation"
create_directories(train_folder, ["images", "labels"])
create_directories(val_folder, ["images", "labels"])

# Copy images and labels to train folder
copy_files_to_folder(train_dataset["images"], os.path.join(train_folder, "images"))
copy_files_to_folder(train_dataset["labels"], os.path.join(train_folder, "labels"))

# Copy images and labels to validation folder
copy_files_to_folder(val_dataset["images"], os.path.join(val_folder, "images"))
copy_files_to_folder(val_dataset["labels"], os.path.join(val_folder, "labels"))

# Print the number of images in the training and validation sets
print("Number of images in training set:", len(train_dataset["images"]))
print("Number of images in validation set:", len(val_dataset["images"]))


updating for yolov7 the class id

In [ ]:
def update_class_ids(label_dir):
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r") as file:
                lines = file.readlines()
                updated_lines = []
                for line in lines:
                    class_id, *rest = map(float, line.strip().split())
                    # Update class IDs
                    if class_id == 4:
                        updated_class_id = 0
                    elif class_id == 5:
                        updated_class_id = 1
                    elif class_id == 7:
                        updated_class_id = 2
                    else:
                        updated_class_id = class_id
                    updated_line = f"{updated_class_id} {' '.join(map(str, rest))}\n"
                    updated_lines.append(updated_line)
            # Write updated labels back to file
            with open(os.path.join(label_dir, filename), "w") as file:
                file.writelines(updated_lines)

# Example usage
label_directory = "data/yolov7-custom/data/train/labels"
update_class_ids(label_directory)
label_directory = "data/yolov7-custom/data/validation/labels"
update_class_ids(label_directory)


In [ ]:
!python data/yolov7-custom/train.py --workers 1 --device cpu --batch-size 8 --epochs 20 --img 640 640 --data data/yolov7-custom/data/custom_data.yaml --hyp data/yolov7-custom/data/hyp.scratch.custom.yaml --cfg data/yolov7-custom/cfg/training/yolov7-custom.yaml --name data/yolov7-custom/yolov7-custom --weights data/yolov7-custom/yolov7.pt
